# How to use SoftPosit.jl - some examples

In [4]:
using SoftPosit

Conversion from Float, Int, bit pattern or hexadecimal

In [10]:
p0 = Posit16(1.23)
Float64(p0)

1.22998046875

In [12]:
p1 = Posit8(50)
Float64(p1)

64.0

(Note that 64 is the closest number in the Posit8,0 system to 50!)

In [15]:
p1_2 = Posit8_2(50)
Float64(p1_2)

48.0

In the Posit8,2 system there is also 48, thanks to two exponent bits!

In [5]:
p0 = Posit8(0b10100101)

Posit8(0xa5)

In [8]:
p1 = Posit32(0x6bb73333)

Posit32(0x6bb73333)

In [6]:
bitstring(p)

"10100101"

In [7]:
bitstring(p," ")

"1 01 00101"